<a href="https://colab.research.google.com/github/Rogerio-mack/data-engineering/blob/main/Aula_kafka_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/Rogerio-mack/data-engineering/blob/main/mack_logo.png?raw=true" height="70" align="right"/></a>






# Kafka Docker

# Start the Kafka Cluster with Docker Compose

In [ ]:
version: '3'
services:
  zookeeper:
    image: wurstmeister/zookeeper
    container_name: zookeeper
    ports:
      - "2181:2181"
    networks:
      - kafka-net
  kafka:
    image: wurstmeister/kafka
    container_name: kafka
    ports:
      - "9092:9092"
    environment:
      KAFKA_ADVERTISED_LISTENERS: INSIDE://kafka:9092,OUTSIDE://localhost:9093
      KAFKA_LISTENER_SECURITY_PROTOCOL_MAP: INSIDE:PLAINTEXT,OUTSIDE:PLAINTEXT
      KAFKA_LISTENERS: INSIDE://0.0.0.0:9092,OUTSIDE://0.0.0.0:9093
      KAFKA_INTER_BROKER_LISTENER_NAME: INSIDE
      KAFKA_ZOOKEEPER_CONNECT: zookeeper:2181
    networks:
      - kafka-net
networks:
  kafka-net:
    driver: bridge

In [ ]:
docker-compose up -d

```
rogerio@DESKTOP-KRMG5G5:/$ docker-compose up -d
Recreating kafka ...
Starting zookeeper ...

Starting zookeeper ... done
```

# Verifying

Pelos comandos abaixo ou através do Docker-Desktop.

In [ ]:
docker-compose ps

```
rogerio@DESKTOP-KRMG5G5:/$ docker ps
CONTAINER ID   IMAGE                    COMMAND                  CREATED          STATUS              PORTS
                                   NAMES
9529ff3d726f   wurstmeister/kafka       "start-kafka.sh"         31 minutes ago   Up 10 seconds       0.0.0.0:9092->9092/tcp                               9529ff3d726f_kafka
3cf07df3f902   wurstmeister/zookeeper   "/bin/sh -c '/usr/sb…"   31 minutes ago   Up About a minute   22/tcp, 2888/tcp, 3888/tcp, 0.0.0.0:2181->2181/tcp   zookeeper
```

### Check logs

In [ ]:
docker-compose logs zookeeper | grep -i binding

```
rogerio@DESKTOP-KRMG5G5:/$ docker-compose logs zookeeper | grep -i binding
zookeeper    | 2024-08-20 20:04:46,222 [myid:] - INFO  [main:NIOServerCnxnFactory@89] - binding to port 0.0.0.0/0.0.0.0:2181
zookeeper    | 2024-08-20 20:33:34,347 [myid:] - INFO  [main:NIOServerCnxnFactory@89] - binding to port 0.0.0.0/0.0.0.0:2181
```

In [ ]:
docker-compose logs kafka | grep -i started

```
rogerio@DESKTOP-KRMG5G5:/$ docker-compose logs kafka | grep -i started
9529ff3d726f_kafka | [2024-08-20 20:04:50,019] INFO [SocketServer listenerType=ZK_BROKER, nodeId=1001] Started data-plane acceptor and processor(s) for endpoint : ListenerName(INSIDE) (kafka.network.SocketServer)
9529ff3d726f_kafka | [2024-08-20 20:04:50,022] INFO [SocketServer listenerType=ZK_BROKER, nodeId=1001] Started data-plane acceptor and processor(s) for endpoint : ListenerName(OUTSIDE) (kafka.network.SocketServer)
9529ff3d726f_kafka | [2024-08-20 20:04:50,022] INFO [SocketServer listenerType=ZK_BROKER, nodeId=1001] Started socket server acceptors and processors (kafka.network.SocketServer)
9529ff3d726f_kafka | [2024-08-20 20:04:50,031] INFO [KafkaServer id=1001] started (kafka.server.KafkaServer)
9529ff3d726f_kafka | [2024-08-20 20:35:05,274] INFO [SocketServer listenerType=ZK_BROKER, nodeId=1001] Started data-plane acceptor and processor(s) for endpoint : ListenerName(INSIDE) (kafka.network.SocketServer)
9529ff3d726f_kafka | [2024-08-20 20:35:05,278] INFO [SocketServer listenerType=ZK_BROKER, nodeId=1001] Started data-plane acceptor and processor(s) for endpoint : ListenerName(OUTSIDE) (kafka.network.SocketServer)
9529ff3d726f_kafka | [2024-08-20 20:35:05,278] INFO [SocketServer listenerType=ZK_BROKER, nodeId=1001] Started socket server acceptors and processors (kafka.network.SocketServer)
9529ff3d726f_kafka | [2024-08-20 20:35:05,293] INFO [KafkaServer id=1001] started (kafka.server.KafkaServer)
```

# Creating a Kafka Topic

In [ ]:
docker-compose exec kafka kafka-topics.sh --create --topic mytopic1 --partitions 1 --replication-factor 1 --bootstrap-server kafka:9092

```
rogerio@DESKTOP-KRMG5G5:/$ docker-compose exec kafka kafka-topics.sh --create --topic mytopic1 --partitions 1 --replicat
ion-factor 1 --bootstrap-server kafka:9092
Created topic mytopic1.
```

# Publishing Messages

In [ ]:
docker-compose exec kafka kafka-console-producer.sh --topic mytopic1 --broker-list kafka:9092

```
rogerio@DESKTOP-KRMG5G5:/$ docker-compose exec kafka kafka-console-producer.sh --topic mytopic1 --broker-list kafka:9092
>my_msg1
>my_msg2
>{'id':1,'msg':'teste'}
>{'id':2,'msg':'teste','etc':'...'}
>1
>2
>3
>4
>4
>^Crogerio@DESKTOP-KRMG5G5:/$
```

# Consuming Messages

In [ ]:
docker-compose exec kafka kafka-console-consumer.sh --topic mytopic1 --from-beginning --bootstrap-server kafka:9092

```
rogerio@DESKTOP-KRMG5G5:/$ docker-compose exec kafka kafka-console-consumer.sh --topic mytopic1 --from-beginning --bootstrap-server kafka:9092092
my_msg1
my_msg2
{'id':1,'msg':'teste'}
{'id':2,'msg':'teste','etc':'...'}
1
2
3
4
4
```